<a href="https://www.kaggle.com/parisrohan/black-friday-xgb-regressor?scriptVersionId=88918707" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/black-friday/train.csv
/kaggle/input/black-friday/test.csv


In [2]:
#load train data
train_data=pd.read_csv('/kaggle/input/black-friday/train.csv')
train_data.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     550068 non-null  int64  
 1   Product_ID                  550068 non-null  object 
 2   Gender                      550068 non-null  object 
 3   Age                         550068 non-null  object 
 4   Occupation                  550068 non-null  int64  
 5   City_Category               550068 non-null  object 
 6   Stay_In_Current_City_Years  550068 non-null  object 
 7   Marital_Status              550068 non-null  int64  
 8   Product_Category_1          550068 non-null  int64  
 9   Product_Category_2          376430 non-null  float64
 10  Product_Category_3          166821 non-null  float64
 11  Purchase                    550068 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


In [4]:
#load test data
test_data=pd.read_csv('/kaggle/input/black-friday/test.csv')
test_data.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [5]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233599 entries, 0 to 233598
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     233599 non-null  int64  
 1   Product_ID                  233599 non-null  object 
 2   Gender                      233599 non-null  object 
 3   Age                         233599 non-null  object 
 4   Occupation                  233599 non-null  int64  
 5   City_Category               233599 non-null  object 
 6   Stay_In_Current_City_Years  233599 non-null  object 
 7   Marital_Status              233599 non-null  int64  
 8   Product_Category_1          233599 non-null  int64  
 9   Product_Category_2          161255 non-null  float64
 10  Product_Category_3          71037 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 19.6+ MB


In [6]:
#merge train and test data to perform data preprocessing
df_merged_data= pd.concat([test_data.assign(ind="test"), train_data.assign(ind="train")])
df_merged_data.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,ind,Purchase
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN,test,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN,test,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN,test,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN,test,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0,test,NaN


In [7]:
df_merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783667 entries, 0 to 550067
Data columns (total 13 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     783667 non-null  int64  
 1   Product_ID                  783667 non-null  object 
 2   Gender                      783667 non-null  object 
 3   Age                         783667 non-null  object 
 4   Occupation                  783667 non-null  int64  
 5   City_Category               783667 non-null  object 
 6   Stay_In_Current_City_Years  783667 non-null  object 
 7   Marital_Status              783667 non-null  int64  
 8   Product_Category_1          783667 non-null  int64  
 9   Product_Category_2          537685 non-null  float64
 10  Product_Category_3          237858 non-null  float64
 11  ind                         783667 non-null  object 
 12  Purchase                    550068 non-null  float64
dtypes: float64(3),

In [8]:
#Get count of missing values in each column
def get_cols_with_missing_values(DataFrame):
    missing_na_columns=(DataFrame.isnull().sum())
    return missing_na_columns[missing_na_columns > 0]

#missing values in merged data - ignore Purchase as it is target
print(get_cols_with_missing_values(df_merged_data))

Product_Category_2    245982
Product_Category_3    545809
Purchase              233599
dtype: int64


In [9]:
#Impute missing values
from sklearn.impute import SimpleImputer as SI
SI_imputer=SI(strategy='median')
df_merged_data['Product_Category_2']= SI_imputer.fit_transform(df_merged_data['Product_Category_2'].to_numpy().reshape(-1,1))
df_merged_data['Product_Category_3']= SI_imputer.fit_transform(df_merged_data['Product_Category_3'].to_numpy().reshape(-1,1))

In [10]:
#missing values  - ignore Purchase as it is target
print(get_cols_with_missing_values(df_merged_data))

Purchase    233599
dtype: int64


In [11]:
df_merged_data.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,ind,Purchase
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,14.0,test,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,14.0,test,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,14.0,test,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,14.0,test,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0,test,NaN


In [12]:
df_merged_data.columns

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'ind', 'Purchase'],
      dtype='object')

In [13]:
#Drop User_ID column
df_merged_data_2 = df_merged_data.drop('User_ID', axis=1)

In [14]:
#Encode Gender column
df_merged_data_2["Gender"] = df_merged_data_2["Gender"].map({"F":0, "M":1})
df_merged_data_2.head()

,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,ind,Purchase
0,P00128942,1,46-50,7,B,2,1,1,11.0,14.0,test,NaN
1,P00113442,1,26-35,17,C,0,0,3,5.0,14.0,test,NaN
2,P00288442,0,36-45,1,B,4+,1,5,14.0,14.0,test,NaN
3,P00145342,0,36-45,1,B,4+,1,4,9.0,14.0,test,NaN
4,P00053842,0,26-35,1,C,1,0,4,5.0,12.0,test,NaN


In [15]:
df_merged_data_2.Age.unique()

array(['46-50', '26-35', '36-45', '18-25', '51-55', '55+', '0-17'],
      dtype=object)

In [16]:
#Encode Age column
df_merged_data_2["Age"] = df_merged_data_2["Age"].map({"0-17":1, "18-25":2, "26-35":3, "36-45":4, "46-50":5, "51-55":6, "55+":7 })
df_merged_data_2.head()

,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,ind,Purchase
0,P00128942,1,5,7,B,2,1,1,11.0,14.0,test,NaN
1,P00113442,1,3,17,C,0,0,3,5.0,14.0,test,NaN
2,P00288442,0,4,1,B,4+,1,5,14.0,14.0,test,NaN
3,P00145342,0,4,1,B,4+,1,4,9.0,14.0,test,NaN
4,P00053842,0,3,1,C,1,0,4,5.0,12.0,test,NaN


In [17]:
df_merged_data_2.City_Category.unique()

array(['B', 'C', 'A'], dtype=object)

In [18]:
#Encode city category - drop first column as two column suffice to represent three values
city = pd.get_dummies(df_merged_data_2["City_Category"], drop_first = True)
city.head()

,B,C
0,1,0
1,0,1
2,1,0
3,1,0
4,0,1


In [19]:
df_merged_data_2 = pd.concat([df_merged_data_2, city], axis = 1)
df_merged_data_2.drop("City_Category", axis = 1, inplace = True)
df_merged_data_2.head()

,Product_ID,Gender,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,ind,Purchase,B,C
0,P00128942,1,5,7,2,1,1,11.0,14.0,test,NaN,1,0
1,P00113442,1,3,17,0,0,3,5.0,14.0,test,NaN,0,1
2,P00288442,0,4,1,4+,1,5,14.0,14.0,test,NaN,1,0
3,P00145342,0,4,1,4+,1,4,9.0,14.0,test,NaN,1,0
4,P00053842,0,3,1,1,0,4,5.0,12.0,test,NaN,0,1


In [20]:
df_merged_data_2.Stay_In_Current_City_Years.unique()

array(['2', '0', '4+', '1', '3'], dtype=object)

In [21]:
#Encode Stay column
df_merged_data_2["Stay_In_Current_City_Years"] = df_merged_data_2["Stay_In_Current_City_Years"].map({"0":0, "1":1, "2":2, "3":3, "4+":4})
df_merged_data_2.head()

,Product_ID,Gender,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,ind,Purchase,B,C
0,P00128942,1,5,7,2,1,1,11.0,14.0,test,NaN,1,0
1,P00113442,1,3,17,0,0,3,5.0,14.0,test,NaN,0,1
2,P00288442,0,4,1,4,1,5,14.0,14.0,test,NaN,1,0
3,P00145342,0,4,1,4,1,4,9.0,14.0,test,NaN,1,0
4,P00053842,0,3,1,1,0,4,5.0,12.0,test,NaN,0,1


In [22]:
df_merged_data_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783667 entries, 0 to 550067
Data columns (total 13 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Product_ID                  783667 non-null  object 
 1   Gender                      783667 non-null  int64  
 2   Age                         783667 non-null  int64  
 3   Occupation                  783667 non-null  int64  
 4   Stay_In_Current_City_Years  783667 non-null  int64  
 5   Marital_Status              783667 non-null  int64  
 6   Product_Category_1          783667 non-null  int64  
 7   Product_Category_2          783667 non-null  float64
 8   Product_Category_3          783667 non-null  float64
 9   ind                         783667 non-null  object 
 10  Purchase                    550068 non-null  float64
 11  B                           783667 non-null  uint8  
 12  C                           783667 non-null  uint8  
dtypes: float64(3),

In [23]:
#convert dtype to int
df_merged_data_2["B"] = df_merged_data_2["B"].astype(int)
df_merged_data_2["C"] = df_merged_data_2["C"].astype(int)

In [24]:
df_merged_data_2 = df_merged_data_2.drop('Product_ID', axis=1)

In [25]:
df_merged_data_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783667 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Gender                      783667 non-null  int64  
 1   Age                         783667 non-null  int64  
 2   Occupation                  783667 non-null  int64  
 3   Stay_In_Current_City_Years  783667 non-null  int64  
 4   Marital_Status              783667 non-null  int64  
 5   Product_Category_1          783667 non-null  int64  
 6   Product_Category_2          783667 non-null  float64
 7   Product_Category_3          783667 non-null  float64
 8   ind                         783667 non-null  object 
 9   Purchase                    550068 non-null  float64
 10  B                           783667 non-null  int64  
 11  C                           783667 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 77.7+ MB


In [26]:
#Split train and test data based on 'ind' column
test, train = df_merged_data_2[df_merged_data_2["ind"].eq("test")], df_merged_data_2[df_merged_data_2["ind"].eq("train")]

In [27]:
test = test.drop('ind', axis=1)
test = test.drop('Purchase', axis=1)
test.head()

,Gender,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,B,C
0,1,5,7,2,1,1,11.0,14.0,1,0
1,1,3,17,0,0,3,5.0,14.0,0,1
2,0,4,1,4,1,5,14.0,14.0,1,0
3,0,4,1,4,1,4,9.0,14.0,1,0
4,0,3,1,1,0,4,5.0,12.0,0,1


In [28]:
train = train.drop('ind', axis=1)
train.head()

,Gender,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,B,C
0,0,1,10,2,0,3,9.0,14.0,8370.0,0,0
1,0,1,10,2,0,1,6.0,14.0,15200.0,0,0
2,0,1,10,2,0,12,9.0,14.0,1422.0,0,0
3,0,1,10,2,0,12,14.0,14.0,1057.0,0,0
4,1,7,16,4,0,8,9.0,14.0,7969.0,0,1


In [29]:
X= train.loc[:, train.columns!='Purchase']
y=train['Purchase']

In [30]:
# Train and test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                test_size = 0.25, random_state = 0)

In [31]:
from xgboost import XGBRegressor as XGBR
model=XGBR(n_estimators=1000, learning_rate=0.05)
model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_test, y_test)], verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [32]:
from sklearn.metrics import mean_absolute_error as mae
pred=model.predict(X_test)
print(mae(y_test, pred))

2150.525163915912


In [33]:
#Get test predictions
pred_test=model.predict(test)

In [34]:
# Save predictions in format used for competition scoring
output = pd.DataFrame({'Id': test.index,
                       'SalePrice': pred_test})
output.to_csv('submission.csv', index=False)